In [33]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import dotenv
import os
from sentence_transformers import SentenceTransformer
import sys

model = SentenceTransformer('Snowflake/snowflake-arctic-embed-l', trust_remote_code=True)

api_key = os.getenv("PINECONE_API_KEY")
env = os.getenv("PINECONE_ENV")

# Get the current working directory
cwd = os.getcwd()

# Add the '../scripts' directory to the system path
sys.path.insert(0, os.path.abspath(os.path.join(cwd, '../scripts')))
sys.path.insert(0, os.path.abspath(os.path.join(cwd, '../src')))
sys.path.insert(0, os.path.abspath(os.path.join(cwd, '../src/data_chunker')))



pc = Pinecone(api_key=api_key, environment=env)

You try to use a model that was created with version 2.7.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





ModuleNotFoundError: No module named 'recursive_chunker'

In [24]:
from pinecone_text.sparse import BM25Encoder
bm25 = BM25Encoder()

df_subset = pd.read_csv('../../assets/csv/data_subset.csv', index_col=0)
df_subset.head()

,id,url,last_updated,html_content,text
344,56f56a63650a002093c08e80893d0507,https://www.wiwi.hu-berlin.de/en/Professorship...,2021-11-10,"<!DOCTYPE html>\n<html xmlns=""http://www.w3.or...",course / short description General Management ...
92,83c06029a4d3f4df99d92f8db3036e69,https://www.wiwi.hu-berlin.de/en/academic-care...,2024-06-11,"<!DOCTYPE html>\n<html xmlns=""http://www.w3.or...","During the doctoral studies, there are various..."
514,1686e07fbe044704640183081b10ce89,https://www.wiwi.hu-berlin.de/en/Professorship...,2024-07-10,"<!DOCTYPE html>\n<html xmlns=""http://www.w3.or...",The Finance Group @ Humboldt consists of those...
82,363616941311823dd3e1b939f2c8b58a,https://www.wiwi.hu-berlin.de/en/academic-care...,2021-11-30,"<!DOCTYPE html>\n<html xmlns=""http://www.w3.or...",NaN
74,3661c6803d347e8d73af15a084bedd1b,https://www.wiwi.hu-berlin.de/en/academic-care...,2022-02-11,"<!DOCTYPE html>\n<html xmlns=""http://www.w3.or...",The formal basis of doctoral studies is consti...


In [26]:
from recursive_chunker import *

ModuleNotFoundError: No module named 'recursive_chunker'

In [12]:
import pandas as pd
qa_df = pd.read_csv('../../assets/csv/qa_df.csv', index_col=0)
question = qa_df['question'].iloc[0]
question_embedded = model.encode(question)
question_embedded

array([ 0.00893313, -0.05035462,  0.00422336, ...,  0.02453731,
       -0.01775656, -0.02322396], dtype=float32)

In [55]:
chunk_stats, char_df = process_data_tokens(df_subset,[2000])
char_df = pd.DataFrame(char_df)
char_df.head(5)

,unique_id,url,last_updated,html_content,text,len,general_id
0,56f56a63650a002093c08e80893d0507_1,https://www.wiwi.hu-berlin.de/en/Professorship...,2021-11-10,"<!DOCTYPE html>\n<html xmlns=""http://www.w3.or...",course / short description General Management ...,952.0,56f56a63650a002093c08e80893d0507
1,83c06029a4d3f4df99d92f8db3036e69_1,https://www.wiwi.hu-berlin.de/en/academic-care...,2024-06-11,"<!DOCTYPE html>\n<html xmlns=""http://www.w3.or...","During the doctoral studies, there are various...",1612.0,83c06029a4d3f4df99d92f8db3036e69
2,1686e07fbe044704640183081b10ce89_1,https://www.wiwi.hu-berlin.de/en/Professorship...,2024-07-10,"<!DOCTYPE html>\n<html xmlns=""http://www.w3.or...",The Finance Group @ Humboldt consists of those...,2000.0,1686e07fbe044704640183081b10ce89
3,1686e07fbe044704640183081b10ce89_2,https://www.wiwi.hu-berlin.de/en/Professorship...,2024-07-10,"<!DOCTYPE html>\n<html xmlns=""http://www.w3.or...","Thu: 10 - 15 Brown, Felicia DOR 1, 304 2093 -...",1017.0,1686e07fbe044704640183081b10ce89
5,3661c6803d347e8d73af15a084bedd1b_1,https://www.wiwi.hu-berlin.de/en/academic-care...,2022-02-11,"<!DOCTYPE html>\n<html xmlns=""http://www.w3.or...",The formal basis of doctoral studies is consti...,2000.0,3661c6803d347e8d73af15a084bedd1b


In [68]:
char_df['text'].iloc[0]

bm25.fit(char_df['text'].iloc[0])
bm25.encode_documents(char_df['text'].iloc[0])

import tqdm
from typing import List, Tuple, Any

def get_embeddings(embed_model: Any, texts: List[str]) -> List[List[float]]:
    """
    A wrapper function to get embeddings from different types of models.

    Args:
        embed_model (Any): The embedding model (either HuggingFaceEmbeddings or a model with encode_documents method).
        texts (List[str]): A list of texts to embed.

    Returns:
        List[List[float]]: A list of embeddings.
    """
    if hasattr(embed_model, 'embed_documents'):
        return embed_model.embed_documents(texts)
    elif hasattr(embed_model, 'encode'):
        return embed_model.encode(texts)
    else:
        raise AttributeError("The provided model doesn't have 'embed_documents' or 'encode' method.")


def embed_dataframe(data: pd.DataFrame, embed_model: Any) -> pd.DataFrame:
    """
    Generates embeddings for the text content of each row in the input DataFrame.

    Args:
        data (pd.DataFrame): Input DataFrame containing at least a 'text' column and a
                             'unique_id' column for error reporting.
        embed_model (Any): An embedding model object that has either 'embed_documents' or 'encode' method.

    Returns:
        pd.DataFrame: A new DataFrame with an additional 'embedding' column containing
                      the generated embeddings. Rows where embedding failed are removed.
    """
    embeddings = []
    for _, row in tqdm(data.iterrows(), total=len(data), desc="Embedding texts"):
        try:
            embedding = get_embeddings(embed_model, [row['text']])[0]
            embeddings.append(embedding)
        except Exception as e:
            
            embeddings.append(None)
    
    data['embedding'] = embeddings
    result_df = data.dropna(subset=['embedding'])
    
    return result_df


df_subset

embedded = embed_dataframe(df_subset, model)


100%|██████████| 952/952 [00:00<00:00, 35025.50it/s]


TypeError: 'module' object is not callable

In [83]:
import numpy as np
from tqdm import tqdm
import pandas as pd
from collections import Counter

from pinecone_text.sparse import BM25Encoder
bm25 = BM25Encoder()

model = SentenceTransformer('Snowflake/snowflake-arctic-embed-l', trust_remote_code=True)


def embed_dataframe(data: pd.DataFrame, embed_model: Any, sparse_model: Any) -> pd.DataFrame:
    """
    Generates embeddings (both dense and sparse) for the text content of each row in the input DataFrame.

    Args:
        data (pd.DataFrame): Input DataFrame containing at least a 'text' column and a
                             'unique_id' column for error reporting.
        embed_model (Any): An embedding model object that has either 'embed_documents' or 'encode' method.
        sparse_model (Any): A model object that generates sparse embeddings, such as BM25.

    Returns:
        pd.DataFrame: A new DataFrame with additional 'embedding' and 'sparse_embedding' columns containing
                      the generated embeddings. Rows where embedding failed are removed.
    """
    dense_embeddings = []
    sparse_embeddings = []
    
    for _, row in tqdm(data.iterrows(), total=len(data), desc="Embedding texts"):
        try:
            # Get dense embedding
            dense_embedding = get_embeddings(embed_model, [row['text']])[0]
            dense_embeddings.append(dense_embedding)
        except Exception as e:
            print(f"Dense embedding failed for ID {row['unique_id']}: {str(e)}")
            dense_embeddings.append(np.nan)  # Handle dense embedding failure

        try:
            # Get sparse embedding
            sparse_embedding = dict(Counter(sparse_model.encode_documents([row['text']]))[0]
            sparse_embeddings.append(sparse_embedding)
        except Exception as e:
            print(f"Sparse embedding failed for ID {row['unique_id']}: {str(e)}")
            sparse_embeddings.append(np.nan)  # Handle sparse embedding failure

    # Add the embeddings to the DataFrame
    data['dense_embedding'] = dense_embeddings
    data['sparse_embedding'] = sparse_embeddings
    
    # Drop rows where either embedding failed (i.e., embeddings are NaN)
    result_df = data.dropna(subset=['dense_embedding', 'sparse_embedding'])

    return result_df

# Usage Example
embedded_char = embed_dataframe(char_df, model, bm25)

Embedding texts:   0%|          | 1/537 [00:00<03:51,  2.31it/s]

Sparse embedding failed for ID 56f56a63650a002093c08e80893d0507_1: unhashable type: 'dict'


Embedding texts:   1%|          | 3/537 [00:00<02:28,  3.59it/s]

Sparse embedding failed for ID 83c06029a4d3f4df99d92f8db3036e69_1: unhashable type: 'dict'
Sparse embedding failed for ID 1686e07fbe044704640183081b10ce89_1: unhashable type: 'dict'


Embedding texts:   1%|          | 5/537 [00:01<01:47,  4.97it/s]

...

Embedding texts:   9%|▊         | 46/537 [00:07<01:29,  5.49it/s]

Sparse embedding failed for ID 5cbc7a0eae028d9788258a9cd38f7305_7: unhashable type: 'dict'
Sparse embedding failed for ID 5cbc7a0eae028d9788258a9cd38f7305_8: unhashable type: 'dict'


Embedding texts:   9%|▉         | 49/537 [00:08<01:20,  6.06it/s]

Sparse embedding failed for ID 3b6c223c6c74109b6b905ff96bec92a1_1: unhashable type: 'dict'
Sparse embedding failed for ID 8aaf51e313f7b593d2b4106fa21037a5_1: unhashable type: 'dict'


KeyboardInterrupt: 

In [82]:
from collections import Counter
test = embedded_char['sparse_embedding'].iloc[0]

def build_dict(input_batch):
 # store a batch of sparse embeddings
   sparse_emb = []
   # iterate through input batch
   for token_ids in input_batch:
       indices = []
       values = []
       # convert the input_ids list to a dictionary of key to frequency values
       d = dict(Counter(token_ids))
       for idx in d:
            indices.append(idx)
            values.append(d[idx])
       sparse_emb.append({'indices': indices, 'values': values})
   # return sparse_emb list
   return sparse_emb

build_dict(test)


[{'indices': ['i', 'n', 'd', 'c', 'e', 's'], 'values': [2, 1, 1, 1, 1, 1]},
 {'indices': ['v', 'a', 'l', 'u', 'e', 's'], 'values': [1, 1, 1, 1, 1, 1]}]

In [76]:
def generate_documents(df: pd.DataFrame, chunk_size: int, doc_type: str) -> List[Dict[str, Any]]:
    """
    Generates a list of document dictionaries from the input DataFrame.
    
    Args:
        df (pd.DataFrame): Input DataFrame containing embedded text chunks.
        chunk_size (int): The chunk size used in chunking.
        doc_type (str): The type of document ('sentence', 'token', or 'semantic').
    
    Returns:
        List[Dict[str, Any]]: A list of document dictionaries.
    """
    documents = []
    for _, row in df.iterrows():
        # Debugging: Print row contents
        #print(f"Processing row: {row}")
        
        # Use get() method with a default value to avoid KeyError
        unique_id = row.get('unique_id', str(uuid.uuid4()))
        
        # Prioritize 'id' over 'general_id' for consistency
        general_id = row.get('id', row.get('general_id', 'unknown'))
        
        # Debugging: Print ID fields
        #print(f"unique_id: {unique_id}, general_id: {general_id}")
        
        document = {
            "unique_id": unique_id,
            'values': row.get("embedding", []),
            "metadata": {
                "url": row.get("url", ""),
                "text": row.get("text", ""),
                'general_id': general_id,
                'chunk_size': chunk_size,
                'doc_type': doc_type
            }
        }
        
        # Add 'last_updated' to metadata if it exists in the DataFrame
        if 'last_updated' in row:
            document['metadata']['date'] = row['last_updated']
        
        documents.append(document)
    
    return documents

,unique_id,url,last_updated,html_content,text,len,general_id,dense_embedding,sparse_embedding
0,56f56a63650a002093c08e80893d0507_1,https://www.wiwi.hu-berlin.de/en/Professorship...,2021-11-10,"<!DOCTYPE html>\n<html xmlns=""http://www.w3.or...",course / short description General Management ...,952.0,56f56a63650a002093c08e80893d0507,"[-0.0006726693, -0.047064904, -0.00024737045, ...","{'indices': [192565064, 883390095, 3071667970,..."
1,83c06029a4d3f4df99d92f8db3036e69_1,https://www.wiwi.hu-berlin.de/en/academic-care...,2024-06-11,"<!DOCTYPE html>\n<html xmlns=""http://www.w3.or...","During the doctoral studies, there are various...",1612.0,83c06029a4d3f4df99d92f8db3036e69,"[-0.007625542, -0.027227053, 0.035203204, -0.0...","{'indices': [1334991813, 4144206424, 215942184..."
2,1686e07fbe044704640183081b10ce89_1,https://www.wiwi.hu-berlin.de/en/Professorship...,2024-07-10,"<!DOCTYPE html>\n<html xmlns=""http://www.w3.or...",The Finance Group @ Humboldt consists of those...,2000.0,1686e07fbe044704640183081b10ce89,"[0.043651793, -0.026485657, 0.023746977, 0.003...","{'indices': [3310569130, 1021187622, 315661618..."
3,1686e07fbe044704640183081b10ce89_2,https://www.wiwi.hu-berlin.de/en/Professorship...,2024-07-10,"<!DOCTYPE html>\n<html xmlns=""http://www.w3.or...","Thu: 10 - 15 Brown, Felicia DOR 1, 304 2093 -...",1017.0,1686e07fbe044704640183081b10ce89,"[-0.00072776823, -0.10089667, -0.0037318603, -...","{'indices': [1045259396, 2263091519, 359766348..."
5,3661c6803d347e8d73af15a084bedd1b_1,https://www.wiwi.hu-berlin.de/en/academic-care...,2022-02-11,"<!DOCTYPE html>\n<html xmlns=""http://www.w3.or...",The formal basis of doctoral studies is consti...,2000.0,3661c6803d347e8d73af15a084bedd1b,"[0.004780789, -0.05352055, 0.04896739, 0.01004...","{'indices': [3168814557, 1772413527, 133499181..."
